# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122620e+02     1.717039e+00
 * time: 0.11258101463317871
     1     1.086243e+01     9.062864e-01
 * time: 0.9429678916931152
     2    -1.184163e+01     1.069249e+00
 * time: 1.081610918045044
     3    -3.421049e+01     7.859519e-01
 * time: 1.233525037765503
     4    -4.789936e+01     5.349982e-01
 * time: 1.3769338130950928
     5    -5.702937e+01     2.121210e-01
 * time: 1.5203218460083008
     6    -5.991042e+01     8.179866e-02
 * time: 1.6257529258728027
     7    -6.087418e+01     6.779837e-02
 * time: 1.7382149696350098
     8    -6.131757e+01     5.481052e-02
 * time: 1.8504698276519775
     9    -6.163844e+01     3.485326e-02
 * time: 1.9558699131011963
    10    -6.187384e+01     2.580575e-02
 * time: 2.0719618797302246
    11    -6.200647e+01     2.052913e-02
 * time: 2.207824945449829
    12    -6.208342e+01     1.975253e-02
 * time: 2.3137118816375732
    13    -6.213232e+01     1.801704e-02
 * time: 2.425880908966

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671064
    AtomicLocal         -18.8557695
    AtomicNonlocal      14.8522665
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485373 
    Xc                  -19.3336820

    total               -62.261666460656
